In [2]:
import segmentation_models_pytorch as smp
import torch

EPS = 1e-10


class SKeMaModel(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.model = smp.Unet(
            encoder_name="tu-maxvit_tiny_tf_512",
            in_channels=10,
            encoder_weights=None,
        )

        self.register_buffer(
            "per_channel_mean",
            torch.tensor([
                1.93357159e02,
                2.53693333e02,
                1.41648022e02,
                9.99292362e02,
                3.21693919e02,
                6.49704998e-02,
                1.57273007e-01,
                -1.57273007e-01,
                1.82229161e07,
                1.09806622e-01,
            ]).view(1, -1, 1, 1),
        )

        self.register_buffer(
            "per_channel_std",
            torch.tensor([
                1.55697494e02,
                2.12700364e02,
                2.04018106e02,
                1.27588129e03,
                3.77324432e02,
                6.75251176e-01,
                7.32966188e-01,
                7.32966188e-01,
                2.16768826e10,
                4.11232123e-01,
            ]).view(1, -1, 1, 1),
        )

    def forward(self, x):
        # Unpack spectral bands
        blue = x.select(1, 0).unsqueeze(1)
        green = x.select(1, 1).unsqueeze(1)
        red = x.select(1, 2).unsqueeze(1)
        nir = x.select(1, 3).unsqueeze(1)
        re = x.select(1, 4).unsqueeze(1)

        # Compute vegetation indices
        ndvi = self.normalized_index(nir, red)
        gndvi = self.normalized_index(nir, green)
        ndvi_re = self.normalized_index(re, red)

        # Compute other indices
        ndwi = self.normalized_index(green, nir)
        chl_green = (nir / (green + EPS)) - 1  # Chlorophyll Index Green

        # Stack all bands and indices
        x_aug = torch.cat([blue, green, red, nir, re, ndvi, ndwi, gndvi, chl_green, ndvi_re], dim=1)

        x_aug_normalized = (x_aug - self.per_channel_mean) / self.per_channel_std

        return self.model(x_aug_normalized)

    @staticmethod
    def normalized_index(a, b):
        return (a - b) / (a + b + EPS)


model = SKeMaModel()

sample_input = torch.rand((2, 5, 512, 512), device=torch.device("cpu"), requires_grad=False)
model(sample_input)

/Users/taylor.denouden/Documents/PycharmProjects/kelp-o-matic/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tensor([[[[ 0.0673, -0.1038, -0.0036,  ..., -0.4200, -0.3432,  0.2372],
          [-0.3212, -0.6060, -0.3758,  ..., -0.6710, -0.9601, -0.3606],
          [-0.5148, -0.5346, -0.1938,  ..., -0.3407,  0.1450,  0.1428],
          ...,
          [-0.2488, -0.2109,  0.0623,  ...,  0.1308,  0.3180, -0.0263],
          [ 0.0498, -0.6152, -0.0099,  ...,  0.2848,  0.3143, -0.2256],
          [-0.3968, -0.5310, -0.3592,  ...,  0.4874, -0.0841, -0.1486]]],


        [[[-0.0040, -0.3440, -0.1445,  ...,  0.1392, -0.4046,  0.0536],
          [-0.6309,  0.0186,  0.3355,  ..., -0.3364, -0.2863, -0.4915],
          [-0.3740, -0.1786,  0.0284,  ..., -0.4980,  0.0996,  0.1463],
          ...,
          [-0.1797,  0.4279,  0.5669,  ..., -0.3368, -0.1825,  0.1968],
          [ 0.0198, -0.0563,  0.3539,  ...,  0.1226, -0.0461,  0.1824],
          [-0.4663, -0.6480,  0.0725,  ..., -0.2525, -0.3959, -0.2123]]]],
       grad_fn=<ConvolutionBackward0>)

In [4]:
ckpt = torch.load("./Unet_tu-maxvit_tiny_tf_512_20250818_164043.ckpt", map_location="cpu")
state_dict = ckpt["state_dict"]

# Update keys
del state_dict["mean"]
del state_dict["std"]
model.load_state_dict(state_dict, strict=False)
model.eval()

SKeMaModel(
  (model): Unet(
    (encoder): TimmUniversalEncoder(
      (model): FeatureListNet(
        (stem): Stem(
          (conv1): Conv2dSame(10, 64, kernel_size=(3, 3), stride=(2, 2))
          (norm1): BatchNormAct2d(
            64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): GELUTanh()
          )
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        )
        (stages_0): MaxxVitStage(
          (blocks): Sequential(
            (0): MaxxVitBlock(
              (conv): MbConvBlock(
                (shortcut): Downsample2d(
                  (pool): AvgPool2dSame(kernel_size=(2, 2), stride=(2, 2), padding=(0, 0))
                  (expand): Identity()
                )
                (pre_norm): BatchNormAct2d(
                  64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
                  (drop): Identity()
                  (act): Identity()
  

In [11]:
torch.onnx.export(
    model,
    sample_input,
    "./Unet_tu-maxvit_tiny_tf_512_20250818_164043.onnx",
    input_names=["input"],
    output_names=["output"],
    export_params=True,
    external_data=False,  # Store model weights in the model file
    opset_version=18,  # ONNX opset version
    do_constant_folding=True,  # Optimize constants
    verbose=False,
    # dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}},
    dynamic_shapes={"x": (torch.export.Dim("batch"), 5, 512, 512)},
    dynamo=True,
)

Applied 41 of general pattern rewrite rules.


ONNXProgram(
    model=
        <
            ir_version=10,
            opset_imports={'': 18},
            producer_name='pytorch',
            producer_version='2.9.0',
            domain=None,
            model_version=None,
        >
        graph(
            name=main_graph,
            inputs=(
                %"input"<FLOAT,[batch,5,512,512]>
            ),
            outputs=(
                %"output"<FLOAT,[batch,1,512,512]>
            ),
            initializers=(
                %"model.encoder.model.stem.conv1.weight"<FLOAT,[64,10,3,3]>{Tensor(...)},
                %"model.encoder.model.stem.conv1.bias"<FLOAT,[64]>{Tensor(...)},
                %"model.encoder.model.stem.conv2.weight"<FLOAT,[64,64,3,3]>{TorchTensor(...)},
                %"model.encoder.model.stem.conv2.bias"<FLOAT,[64]>{TorchTensor(...)},
                %"model.encoder.model.stages_0.blocks.0.conv.pre_norm.weight"<FLOAT,[64]>{TorchTensor(...)},
                %"model.encoder.model.stages_0.blocks.0